In [7]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 22 10:25:23 2017

@author: SRG
"""

import matplotlib.pyplot as plt

from skimage.io import imread
from skimage import data_dir
from skimage.transform import radon, iradon
from scipy.ndimage import zoom
import os

root = 'D:/git/avm_dataset/dataset/hyu_171025/rectified/set1/output/labeled/class_1'

img_file = os.path.join(root, '00000963.jpg')
image = imread(img_file, as_grey=True)

plt.figure(figsize=(8, 8.5))

plt.subplot(221)
plt.title("Original");
plt.imshow(image, cmap=plt.cm.Greys_r)

plt.subplot(222)
projections = radon(image, theta=[75])
plt.plot(projections);
plt.title("Projections at 0, 45, and 90 degrees")
plt.xlabel("Projection position (pixels)");
plt.ylabel("Intensity");

projections = radon(image)
plt.subplot(223)
plt.title("Radon transform/n(Sinogram)");
plt.xlabel("Project angle (deg)");
plt.ylabel("Projection position (pixels)");
plt.imshow(projections)

reconstruction = iradon(projections)
plt.subplot(224)
plt.title("Reconstruction/nfrom sinogram")
plt.imshow(reconstruction, cmap=plt.cm.Greys_r)

plt.subplots_adjust(hspace=0.4, wspace=0.5)
plt.show()

In [22]:
angle = [164, 165]
plt.subplot(221)
plt.title("Original");
plt.imshow(image, cmap=plt.cm.Greys_r)

plt.subplot(222)
projections = radon(image, theta=angle)
plt.plot(projections);
plt.title("Projections at specific degrees")
plt.xlabel("Projection position (pixels)");
plt.ylabel("Intensity");

plt.subplot(223)
reconstruction = iradon(projections, theta=angle)
plt.imshow(reconstruction[76:316,:], cmap=plt.cm.Greys_r)
plt.show()

In [26]:
projections = radon(image)
plt.subplot(111)
plt.title("Radon transform/n(Sinogram)");
plt.xlabel("Project angle (deg)");
plt.ylabel("Projection position (pixels)");
plt.imshow(projections)
plt.show()



In [ ]:
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage import data_dir
from skimage.transform import radon, iradon
from scipy.ndimage import zoom
import os
import numpy as np
import cv2

root = 'D:/git/avm_dataset/dataset/hyu_171025/rectified/set2/output/labeled/class_1'

img_file = os.path.join(root, '00000100.jpg')
image = imread(img_file, as_grey=True)
image = cv2.resize(image,None,fx=0.5,fy=0.5)

projections = radon(image)
off, ang = np.unravel_index(projections.argmax(), projections.shape)

if ang > 90:
    angle = [ang-90, ang]
else:
    angle = [ang, ang+90]
    
plt.subplot(131)
plt.title("Original");
plt.imshow(image, cmap=plt.cm.Greys_r)

plt.subplot(132)
proj = radon(image, theta=angle)
proj[:,0] = (proj[:,0] /proj[:,0].max())*255.0
proj[:,1] = (proj[:,1] /proj[:,1].max())*255.0
plt.plot(proj);
plt.title("Projections at {} degrees".format(ang))
plt.xlabel("Projection position (pixels)");
plt.ylabel("Intensity");

plt.subplot(133)
reconstruction = iradon(proj, theta=angle)
reconstruction[reconstruction<0]=0
#reconstruction /=reconstruction.max()
#reconstruction *=255.0
reconstruction = reconstruction[36:156,:]
plt.imshow(reconstruction.astype(np.uint8), cmap=plt.cm.Greys_r)
plt.show()

In [53]:
reconstruction.shape

(120, 196)

## Execution Time Measure

In [59]:
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage import data_dir
from skimage.transform import radon, iradon
from scipy.ndimage import zoom
import os
import numpy as np
import time
import cv2

root = 'D:/git/avm_dataset/dataset/hyu_171025/rectified/set1/output/labeled/class_1'

img_file = os.path.join(root, '00000950.jpg')
image = imread(img_file, as_grey=True)
image = cv2.resize(image,None,fx=0.5,fy=0.5)

proj = radon(image)
off, ang = np.unravel_index(proj.argmax(), proj.shape)

if ang > 90:
    angle = [ang-90, ang]
else:
    angle = [ang, ang+90]

proj = radon(image, theta=angle)

start_time = time.time()
for i in range(10):
    #proj = radon(image, theta=angle) 
    reconstruction = iradon(proj, theta=angle)
print("{} ms ".format((time.time() - start_time)/10 * 1000))

1.5047073364257812 ms 


In [7]:
2328/180

12.933333333333334

In [60]:
import matplotlib.pyplot as plt

from skimage.io import imread, imsave
from skimage import data_dir
from skimage.transform import radon, iradon
from scipy.ndimage import zoom
import os
import numpy as np
from tqdm import tqdm
import cv2

RESIZE_RATIO = 0.5

for f in range(1,9):
    root = 'D:/git/avm_dataset/dataset/hyu_171025/rectified/set{}/output/labeled/class_1'.format(f)
    output_folder = 'D:/git/avm_dataset/dataset/hyu_171025/rectified/set{}/output/labeled/class_1/radon_resize_0_5'.format(f)
    if os.path.isdir(output_folder)==False:
        os.mkdir(output_folder)

    max_ang = []
    file_list = []
    for file in os.listdir(root):
        if len(file.split('.')) != 2:
            continue
        file_list.append(file)
    
    
    for i in tqdm(range(0,len(file_list),10)):
        img_file = os.path.join(root, file_list[i])
        image = imread(img_file, as_grey=True)
        image = cv2.resize(image,None,fx=RESIZE_RATIO,fy=RESIZE_RATIO)
        
        projections = radon(image)
        off, ang = np.unravel_index(projections.argmax(), projections.shape)
        max_ang.append(ang)

        if ang > 90:
            angle = [ang-90, ang]
        else:
            angle = [ang, ang+90]

        proj = radon(image, theta=angle)
        reconstruction = iradon(proj, theta=angle)

        reconstruction[reconstruction<0]=0
        reconstruction[:,0] /=reconstruction[:,0].max()
        reconstruction[:,0] *=255.0
        reconstruction[:,1] /=reconstruction[:,1].max()
        reconstruction[:,1] *=255.0

        output_file = os.path.join(output_folder, file_list[i])

        imsave(output_file, reconstruction.astype(np.uint8))

    fig, ax = plt.subplots()
    plt.plot(max_ang, '.--')
    plt.title('Randon Transform based Principle Angle Detection')
    plt.xlabel("frame");
    plt.ylabel("max angle [deg]");
    ax.grid(True)
    fig.savefig(os.path.join(output_folder,'max_angle.png'), bbox_inches='tight')

 74%|███████████████████████████████████████████████████████████▏                    | 114/154 [01:13<00:25,  1.55it/s]

KeyboardInterrupt: 